# Setup notes

* Make sure to have a clock visible
* Check network connectivity
* Displays mirrored
* Slides up
* This notebook
  * ~150% zoom
  * Ideally using 3.7-pre because it has better error messages: python-37/bin/jupyter notebook
  * Full screened (F11)
  * Hide header and toolbar
  * Turn on line numbers
  * Kernel → Restart and clear output
* Examples:
  * getaddrinfo: on vorpus.org or blank
  * clear the async/await example and the happy eyeballs (maybe leaving the function prototype to seed things)
* Two terminals (tilix?) with large font and
  * `nc -l -p 12345`
  * `nc -l -p 54321`
* No other windows on the same desktop
* scrolled down to the getaddrinfo example

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
# `getaddrinfo` example

In [3]:
import socket
socket.getaddrinfo("debian.org", "https")

[(<AddressFamily.AF_INET: 2>,
  <SocketKind.SOCK_STREAM: 1>,
  6,
  '',
  ('149.20.4.15', 443)),
 (<AddressFamily.AF_INET: 2>,
  <SocketKind.SOCK_STREAM: 1>,
  6,
  '',
  ('130.89.148.14', 443)),
 (<AddressFamily.AF_INET: 2>,
  <SocketKind.SOCK_STREAM: 1>,
  6,
  '',
  ('128.31.0.62', 443)),
 (<AddressFamily.AF_INET: 2>,
  <SocketKind.SOCK_STREAM: 1>,
  6,
  '',
  ('5.153.231.4', 443)),
 (<AddressFamily.AF_INET6: 10>,
  <SocketKind.SOCK_STREAM: 1>,
  6,
  '',
  ('2001:4f8:1:c::15', 443, 0, 0)),
 (<AddressFamily.AF_INET6: 10>,
  <SocketKind.SOCK_STREAM: 1>,
  6,
  '',
  ('2001:41c8:1000:21::21:4', 443, 0, 0)),
 (<AddressFamily.AF_INET6: 10>,
  <SocketKind.SOCK_STREAM: 1>,
  6,
  '',
  ('2001:67c:2564:a119::148:14', 443, 0, 0))]

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

# Simple proxy demo

In [8]:
import trio

async def proxy_one_way(source, sink):
    while True:
        data = await source.receive_some(1024)
        if not data:
            await sink.send_eof()
            break
        await sink.send_all(data)
        
async def proxy_two_way(a, b):
    async with trio.open_nursery() as nursery:
        nursery.start_soon(proxy_one_way, a, b)
        nursery.start_soon(proxy_one_way, b, a)
        
async def main():
    with trio.move_on_after(10):  # 10 second time limit
        a = await trio.open_tcp_stream("localhost", 12345)
        b = await trio.open_tcp_stream("localhost", 54321)
        async with a, b:
            await proxy_two_way(a, b)
    print("all done!")
        
trio.run(main)

all done!


In [7]:
async def sleepy():
    print("yawn")
    await trio.sleep(1)
    print("woke up")

async def sleepy_twice():
    await sleepy()
    await sleepy()
    
trio.run(sleepy_twice)

yawn
woke up
yawn
woke up


<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
# `trio.Event` example

In [9]:
async def sleeper(event):
    print("sleeper: going to sleep!")
    await trio.sleep(5)
    print("sleeper: woke up! let's tell everyone")
    event.set()
    
async def waiter(event, i):
    print(f"waiter {i}: waiting for the sleeper")
    await event.wait()
    print(f"waiter {i}: received notification!")
    
async def main():
    async with trio.open_nursery() as nursery:
        event = trio.Event()
        nursery.start_soon(sleeper, event)
        nursery.start_soon(waiter, event, 1)
        nursery.start_soon(waiter, event, 2)
        
trio.run(main)

waiter 2: waiting for the sleeper
waiter 1: waiting for the sleeper
sleeper: going to sleep!
sleeper: woke up! let's tell everyone
waiter 2: received notification!
waiter 1: received notification!


<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
# Happy Eyeballs!

In [11]:
async def open_tcp_socket(hostname, port, *, patience=0.250):
    targets = await trio.socket.getaddrinfo(hostname, port, type=trio.socket.SOCK_STREAM)
    failed_attempts = [trio.Event() for _ in targets]
    winning_socket = None
    
    async def attempt(which, nursery):
        # wait for the last one to fail or the timeout
        with trio.move_on_after(patience):
            if which > 0:
                await failed_attempts[which - 1].wait()
        
        # kick off the next attempt
        if which + 1 < len(targets):
            nursery.start_soon(attempt, which + 1, nursery)
        
        # make this attempt
        *socket_config, _, socket_target = targets[which]
        try:
            socket = trio.socket.socket(*socket_config)
            await socket.connect(socket_target)
        # if fails, tell the next one to go ahead
        except OSError:
            failed_attempts[which].set()
        # if succeeds, cancel all other attempts and return winning socket
        else:
            nursery.cancel_scope.cancel()
            nonlocal winning_socket
            winning_socket = socket
    
    async with trio.open_nursery() as nursery:
        nursery.start_soon(attempt, 0, nursery)
    
    if winning_socket is None:
        raise OSError("ruh-oh")
    else:
        return winning_socket
    
async def main():
    return await open_tcp_socket("debian.org", "https")

trio.run(main)

<trio.socket.socket fd=43, family=AddressFamily.AF_INET, type=2049, proto=6, laddr=('172.30.167.174', 33324), raddr=('130.89.148.14', 443)>

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
# Cheat sheets

In [ ]:
async def sleep_one():
    print("I'm tired")
    await trio.sleep(1)
    print("slept!")
    
async def sleep_twice():
    await sleep_one()
    await sleep_one()
    
trio.run(sleep_twice)

In [ ]:
async def open_connection(hostname, port, *, patience_in_seconds=0.250):
    targets = await trio.socket.getaddrinfo(hostname, port, type=trio.socket.SOCK_STREAM)
    attempt_failed = [trio.Event() for _ in targets]
    winning_socket = None
    
    async def attempt_one(which, nursery):
        # wait for previous attempt to fail, or timeout
        if which > 0:
            with trio.move_on_after(patience_in_seconds):
                await attempt_failed[which - 1].wait()

        # kick off next attempt
        if which + 1 < len(targets):
            nursery.start_soon(attempt_one, which + 1, nursery)
        
        # try to connect to our target
        *socket_config, _, target = targets[which]
        try:
            sock = trio.socket.socket(*socket_config)
            await sock.connect(target)
        # if fail, tell next attempt to go ahead
        except OSError:
            attempt_failed[which].set()
        # if succeed, cancel other attempts and save winning socket
        else:
            nursery.cancel_scope.cancel()
            nonlocal winning_socket
            winning_socket = sock
    
    async with trio.open_nursery() as nursery:
        nursery.start_soon(attempt_one, 0, nursery)
    
    if winning_socket is None:
        raise OSError("failed")
    else:
        return winning_socket
    
trio.run(open_connection, "debian.org", "https")

In [ ]:
import trio

async def open_tcp_socket(hostname, port, *, timeout=0.250):
    targets = await trio.socket.getaddrinfo(hostname, port, type=trio.socket.SOCK_STREAM)
    attempt_failed = [trio.Event() for _ in targets]
    winning_socket = None
    
    async def attempt(which, nursery):
        if which > 0:
            with trio.move_on_after(timeout):
                await attempt_failed[which - 1].wait()
                
        if which + 1 < len(targets):
            nursery.start_soon(attempt, which + 1, nursery)
            
        *socket_config, _, socket_target = targets[which]
        try:
            socket = trio.socket.socket(*socket_config)
            await socket.connect(socket_target)
        except OSError:
            attempt_failed[which].set()
        else:
            nursery.cancel_scope.cancel()
            nonlocal winning_socket
            winning_socket = socket
        
    async with trio.open_nursery() as nursery:
        nursery.start_soon(attempt, 0, nursery)
    
    if winning_socket is None:
        raise OSError("failed")
    else:
        return winning_socket
    
async def main():
    print(await open_tcp_socket("vorpus.org", "https"))
    
trio.run(main)